In [1]:
from optlang import Model, Variable, Constraint, Objective
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
import cobra, cloudpickle

Load data

In [2]:
# Load pairwise distance
df_pairdist = pd.read_csv('./output/Step1_pfba_pairwise_distance_iRhto1108.csv', sep='\t', index_col=0)

#### Define model

In [3]:
model = Model(name='mfa_design')

In [4]:
irhto1108 = cobra.io.load_json_model('../yeast_model/yeast_model/build_iRhto/output/rt_draft/rt_draft8C.json')

#### Define variables

In [5]:
cols = df_pairdist.columns.tolist()
colpairs = []
for i in range(0, len(cols)-1):
    for j in range(i+1, len(cols)):
        colpairs.append([cols[i], cols[j]])

y_list = [Variable('y__' + i, type='binary', lb=0, ub=1) for i in cols]
r_list = [Variable('__'.join(['r'] + i), type='continuous', lb=0, ub=1) for i in colpairs]

In [6]:
vars_model = model.variables.keys()
for var in y_list + r_list:
    if var.name not in vars_model:
        model.add(var)
        
y_list = [i.name for i in model.variables if i.name[:3] == 'y__']
r_list = [i.name for i in model.variables if i.name[:3] == 'r__']

#### Define objective

In [7]:
c_vector = []
for r_var in tqdm(r_list, leave=False):
    i,j = r_var.split('__')[1:]
    c_vector.append(df_pairdist.loc[i,j])
    
obj = Objective(sum(c_vector[i] * model.variables[r_list[i]] for i in range(0, len(r_list))))

In [8]:
model.objective = obj

#### Define constraints

In [9]:
# List 1: r_ij <= y_i
cons_list1 = []
name = 'cons1'; count = -1
for r_var in r_list:
    count += 1
    r_ij = model.variables[r_var]
    y_i = model.variables['y__' + r_var.split('__')[1]]
    cons_list1.append(Constraint(r_ij - y_i, ub=0, name='_'.join([name, 'num'+str(count)])))
    
# List 2: r_ij <= y_j
name = 'cons2'; count = -1
cons_list2 = []
for r_var in r_list:
    count += 1
    r_ij = model.variables[r_var]
    y_j = model.variables['y__' + r_var.split('__')[2]]
    cons_list2.append(Constraint(r_ij - y_j, ub=0, name='_'.join([name, 'num'+str(count)])))
    
# List 3: r_ij >= y_i + y_j - 1
name = 'cons3'; count = -1
cons_list3 = []
for r_var in r_list:
    count += 1
    r_ij = model.variables[r_var]
    y_i = model.variables['y__' + r_var.split('__')[1]]
    y_j = model.variables['y__' + r_var.split('__')[2]]
    cons_list3.append(Constraint(r_ij - y_i - y_j, lb=-1, name='_'.join([name, 'num'+str(count)])))

# Constraint 5: y__WT = 1
name = 'cons5'
cons_5 = [Constraint(model.variables['y__WT'], lb=1, ub=1, name=name)]

In [10]:
model.add(cons_list1 + cons_list2 + cons_list3 + cons_5)

#### Save model as pickle

In [11]:
import cloudpickle

In [12]:
with open('./output/Step2_milp_model.pickle', 'wb') as f:
    cloudpickle.dump(model, f)

### Set parameters and solve

In [1]:
from optlang import Model, Variable, Constraint, Objective
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
import cobra, cloudpickle

Load data

In [3]:
irhto1108 = cobra.io.load_json_model('../yeast_model/yeast_model/build_iRhto/output/rt_draft/rt_draft8C.json')
df_pairdist = pd.read_csv('./output/Step1_pfba_pairwise_distance_iRhto1108.csv', sep='\t', index_col=0)

with open('./output/Step2_milp_model.pickle', 'rb') as f:
    model = cloudpickle.load(f)
y_list = [i.name for i in model.variables if i.name[:3] == 'y__']
r_list = [i.name for i in model.variables if i.name[:3] == 'r__']

Run optimization (results: df_solns)

In [4]:
num_muts = 16 # Number of mutants, counting WT as mutants

cols = ['num_muts', 'mutants_set', 'sum_pairwise_dist', 'opt_status']
df_solns = pd.DataFrame(columns=cols)

count = -1
for num_iter in range(2, num_muts+1):
    
    count += 1
    with open('./output/Step2_milp_model.pickle', 'rb') as f:
        model = cloudpickle.load(f)
    
    # Constraint 4: sum(y) = L
    name = 'cons4'
    cons_4 = [Constraint(sum([model.variables[i] for i in y_list]),
                         lb=num_iter, ub=num_iter, name=name)]
    model.add(cons_4)
    
    model.optimize()
    muts = []
    for var in model.variables.values():
        if var.name[:3] == 'y__' and var.primal > 0.8:
            muts.append(var.name.split('__')[1])
    muts = [mut for mut in muts if mut != 'WT']
    df_solns.loc[count, 'num_muts'] = num_iter-1
    df_solns.loc[count, 'mutants_set'] = ','.join(muts)
    df_solns.loc[count, 'sum_pairwise_dist'] = model.objective.value
    df_solns.loc[count, 'opt_status'] = model.status

In [14]:
df_solns.to_csv('./output/Step2_report_solutions.csv', sep='\t', index=None)

In [5]:
df_solns

,num_muts,mutants_set,sum_pairwise_dist,opt_status
0,1,GAPD_c_ko,0.171254,optimal
1,2,"GAPD_c_ko,MDH_c_ko",0.441603,optimal
2,3,"GAPD_c_ko,MDH_c_ko,RPE_c_ko",0.780705,optimal
3,4,"ASPTAi_m_ko,GAPD_c_ko,MDH_c_ko,RPE_c_ko",1.20367,optimal
4,5,"ASPTAi_m_ko,ENO_c_ko,GAPD_c_ko,MDH_c_ko,RPE_c_ko",1.69563,optimal
5,6,"ASPTAi_m_ko,ENO_c_ko,GAPD_c_ko,GND_c_ko,MDH_c_...",2.23145,optimal
6,7,"ASPTA_c_ko,ASPTAi_m_ko,ENO_c_ko,GAPD_c_ko,GND_...",2.81692,optimal
7,8,"ASPTA_c_ko,ASPTAi_m_ko,ENO_c_ko,GAPD_c_ko,GND_...",3.49217,optimal
8,9,"ASPTA_c_ko,ASPTAi_m_ko,ENO_c_ko,GAPD_c_ko,GLUD...",4.14454,optimal
9,10,"ACITL_c_ko,ASPTA_c_ko,ASPTAi_m_ko,ENO_c_ko,GAP...",4.83093,optimal


In [9]:
# Mutant-mutant pair with highest distance (including WT as mutant)
maxval = df_pairdist.max().max()
for mut1 in df_pairdist:
    for mut2 in df_pairdist:
        if df_pairdist.loc[mut1, mut2] > 0.9999*maxval:
            print mut1, mut2, df_pairdist.loc[mut1, mut2]

GAPD_c_ko MDH_c_ko 0.21230239977857515
MDH_c_ko GAPD_c_ko 0.21230239977857515


#### Record mutants into an interpretable format
(result: df_record)

In [10]:
# Highest recorded mutant-mutant distance
mutmut_max = df_pairdist.max().max()
wtmut_max = df_pairdist.WT.max()

cols = ['Appearance', 'Mutant', 'GPR', 'Name', 'Reaction', 'Distance from WT',
        'Distance from mutant (highest value)']
df_record = pd.DataFrame(index=df_solns.index, columns=cols)

already = []
for i in df_record.index:
    muts = df_solns.mutants_set[i].split(',')
    app = 'Set of ' + str(len(muts))
    
    for mut in muts:
        if mut not in already:
            already.append(mut)
            df_record.loc[i, 'Appearance'] = app
            df_record.loc[i, 'Mutant'] = mut[:-3]
            rxn = irhto1108.reactions.get_by_id(mut[:-3])
            df_record.loc[i, 'GPR'] = rxn.gene_reaction_rule
            df_record.loc[i, 'Name'] = rxn.name
            df_record.loc[i, 'Reaction'] = rxn.reaction
            df_record.loc[i, 'Distance from WT'] = df_pairdist.loc['WT', mut] / wtmut_max
            df_record.loc[i, 'Distance from mutant (highest value)'] = \
                df_pairdist[mut][muts].max() / mutmut_max

In [15]:
df_record.to_csv('./output/Step2_record_mutants.csv', sep='\t', index=None)

In [19]:
irhto1108.reactions.PFK

Reaction identifier,PFK_3_c
Name,phosphofructokinase (s7p)
Memory address,0x07f1f9d458fd0
Stoichiometry,"atp_c + s7p_c --> adp_c + h_c + s17bp_c ATP + sedoheptulose 7-phosphate --> ADP + H+ + sedoheptulose 1,7-bisphosphate"
GPR,rt0491 or rt0495 or rt0499
Lower bound,0
Upper bound,1000


In [20]:
for rxn in df_record.Mutant:
    print irhto1108.reactions.get_by_id(rxn).reaction

g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
ru5p__D_c <=> xu5p__D_c
glu__L_m + oaa_m --> akg_m + asp__L_m
2pg_c <=> h2o_c + pep_c
6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c
akg_c + asp__L_c <=> glu__L_c + oaa_c
3pg_c <=> 2pg_c
akg_c + h_c + nadph_c + nh4_c --> glu__L_c + h2o_c + nadp_c
atp_c + cit_c + coa_c --> accoa_c + adp_c + oaa_c + pi_c
atp_c + f6p_c --> adp_c + fdp_c + h_c
atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
3pg_c + nad_c --> 3php_c + h_c + nadh_c
h2o_c + pser__L_c --> pi_c + ser__L_c
atp_c + s7p_c --> adp_c + h_c + s17bp_c


In [18]:
# For making escher csv file
def make_escher_csv(mflux, path):
    import csv
    import pandas
    if isinstance(mflux, pandas.core.series.Series) != True:
        mflux = mflux.fluxes
    with open(path, 'w') as f:
        fcsv = csv.writer(f, delimiter=',')
        fcsv.writerow(['Rxn', 'Flux'])
        for rxn in mflux.index:
            fcsv.writerow([rxn, mflux[rxn]])